In [1]:
import numpy as np
from scipy.linalg import block_diag
import sympy as sym
import casadi as cas
from casadi import *

The Dugoff model for a typical tire is Dugoff 1970, Wong, 1978

$$
\begin{aligned}
&F_x=-\frac{C_s s}{1-s} f(\lambda) \\
&F_y=-\frac{C_\alpha \tan \alpha}{1-s} f(\lambda)
\end{aligned}
$$
\\
$$
\begin{aligned}
&\lambda=\frac{\mu F_z\left(1-\epsilon v \sqrt{s^2+\tan ^2 \alpha}\right)(1-s)}{2 \overline{c_s^2 s^2+C_\alpha^2 \tan ^2 \alpha}} \\
\\
&f(\lambda)= \begin{cases}X(2-\lambda) & \text { for } \lambda<1 \\
1 & \text { for } \lambda>1\end{cases}
\end{aligned}
$$

$F_x=$ longitudinal force \\
$F_y=$ side force

$\mathrm{s}=$ longitudinal slip \\
$\alpha=$ tire slip angle \\
$C_s=$ cornering stiffness \\
$C_\alpha=$ longitudinal stiffens \\
$F_z=$ vertical load \\
$v=$ vehicle speed \\
$\mu=$ coefficient of road adhesion \\
$\epsilon=$ adhesion reduction coefficient

In this section, we use $\alpha$ to denote the tire slip angle


============================================================================

MODEL USED

given by (Guntur and Sankar, 1980, Dugoff, 1969)
$$
F_x=C_\sigma \frac{\sigma_x}{1+\sigma_x} f(\lambda)
$$
and the lateral tire force is given by $\quad F_y=C_\alpha \frac{\tan (\alpha)}{1+\sigma_x} f(\lambda)$
where $\lambda$ is given by
$$
\lambda=\frac{\mu F_z\left(1+\sigma_x\right)}{2\left\{\left(C_\sigma \sigma_x\right)^2+\left(C_\alpha \tan (\alpha)\right)^2\right\}^{1 / 2}}
$$
and
$f(\lambda)=(2-\lambda) \lambda$ if $\lambda<1$
$f(\lambda)=1$ if $\lambda \geq 1$
$F_z$ is the vertical force on the tire while $\mu$ is the tire-road friction coefficient.

In [2]:
def DugoffLinearizer(slip_angle,slip_ratio, ca, cs, mu, Fz):
  lamb_bar = mu*Fz*(1+slip_ratio)/(2*((cs*slip_ratio)**2+(ca*slip_angle)**2)**0.5)
  if slip_angle !=0 or slip_ratio !=0:
    if lamb_bar < 1:
      fbar = lamb_bar*(2-lamb_bar)
      c_bar = (Fz*ca*mu)/(2*(slip_angle**2*ca**2 + cs**2*slip_ratio**2)**(1/2)) - (Fz*slip_angle**2*ca**3*mu)/(2*(slip_angle**2*ca**2 + cs**2*slip_ratio**2)**(3/2))

    else:
      fbar = 1
      c_bar = ca/(1+slip_ratio)
    Fy_bar= ca*slip_angle*fbar/(1+slip_ratio)
    c_bar = c_bar.item()
    Fy_bar = Fy_bar.item()
  else:
    fbar = 1
    c_bar = ca
    Fy_bar = 0

  return Fy_bar, c_bar, slip_angle

**Optimal_Control_partI.pdf**
2. Linear Quadratic Optimal Control
2.1 Batch Approach Solution

Starting with $x_0=x(0)$, we have $x_1=A x(0)+B u_0$, and $x_2=A x_1+B u_1=A^2 x(0)+A B u_0+B u_1$, by substitution for $x_1$, and so on. Continuing up to $x_N$ we obtain:
$$
\left[\begin{array}{c}
x_0 \\
x_1 \\
\vdots \\
\vdots \\
x_N
\end{array}\right]=\left[\begin{array}{c}
I \\
A \\
\vdots \\
\vdots \\
A^N
\end{array}\right] x(0)+\left[\begin{array}{cccc}
0 & \cdots & \cdots & 0 \\
B & 0 & \cdots & 0 \\
A B & B & \cdots & 0 \\
\vdots & \ddots & \ddots & 0 \\
A^{N-1} B & \cdots & A B & B
\end{array}\right]\left[\begin{array}{c}
u_0 \\
u_1 \\
\vdots \\
\vdots \\
u_{N-1}
\end{array}\right]
$$

The equation above can be represented as
$$
\mathcal{X} \triangleq \mathcal{S}^x x(0)+\mathcal{S}^u U_0
$$

In [3]:
def Sx_Su_Sw_Sd_Sf(A, B, E, D, F, N):
  nX = np.size(A,0)
  nU = np.size(B,1)
  nE = np.size(E,1)

  Sx = np.eye(nX)
  A_tmp = A
  for i in range(N):
      Sx = np.vstack((Sx, A_tmp))
      A_tmp = A_tmp @ A
  
  SxB = Sx @ B
  Su = np.zeros((nX*(N+1),nU*N))
  for j in range(N):
      Su_tmp = np.vstack((np.zeros((nX, nU)), SxB[:-nX,:]))
      Su[:, j:j+nU] = Su_tmp.reshape(Su_tmp.shape[0], nU)
      SxB = Su_tmp
  
  SxE = Sx @ E
  Sw = np.zeros((nX*(N+1),nE*N))
  for k in range(N):
      Sw_tmp = np.vstack((np.zeros((nX, nE)), SxE[:-nX,:]))
      Sw[:, k:k+nE] = Sw_tmp.reshape(Sw_tmp.shape[0], nE)
      SxE = Sw_tmp
  
  Sd = np.zeros((nX,1)).reshape((nX,1))
  D_tmp = D.reshape(nX,1)
  for i in range(N):
      Sd = np.vstack((Sd, D_tmp))
      D_tmp = A @ D_tmp
      
  Sxf = Sx @ F
  Sf = np.zeros((nX*(N+1),N))
  for j in range(N):
      Sf_tmp = np.vstack((np.zeros((nX, 1)), Sxf[:-nX,:]))
      Sf[:, j] = Sf_tmp.reshape(Sf_tmp.shape[0], )
      Sxf = Sf_tmp
  return Sx, Su, Sw, Sd, Sf

In [9]:
def CFTOC_VD(slip_angle, slip_ratio, ca, cs, mux,muy, fz0,fy0, u,reff,delta1,tf,tr,lf,lr,ms,hs,m,ixx,ks,ls,g,bs,iz,kus_d,rmax,Ts,Q,R,W0,Phir,alphamax,Qmax,deltamax,x0,psid,yd,N):
  """
  STATES:
  y: lateral position
  vy: lateral speed
  psi: heading angle
  r: yaw rate
  phi: roll angle
  phid: roll rate
  
  =================================================================
  =================================================================
  Input Definitions:

  slip_angle: tire slip angle at each tire (aka alpha) --- (4x1)
  slip_ratio: tire slip ratio at each tire (aka sigma) --- (4x1)
  ca: tire cornering stifness (aka C_alpha)            --- (4x1)
  cs: tire longitudinal stiffness (aka C_sigma)        --- (4x1)
  mux: coefficient of friction in x                    --- (scalar)
  muy: coefficient of friction in y                    --- (scalar)
  fz0: tire vertical load                              --- (4x1)
  fyo: tire lateral load                               --- (4x1)
  u: longitudinal velcoity                             --- (scalar)
  reff: tire effective radius of rotation              --- (scalar)
  delta1: initial commanded steering input             --- (scalar)
  tf: front wheel track (dist between tire centers)    --- (scalar)
  tr: rear wheel track (dist between tire centers)     --- (scalar)
  lf: distance from cg to front wheels                 --- (scalar)
  lr: distance from cg to rear wheels                  --- (scalar)
  ms: vehicle sprung mass (without mass of tires and suspension) --- (scalar)
  hs: cg of sprung mass to roll center                 --- (scalar)
  m: total mass of vehicle (sprung and unsprung)       --- (scalar)
  ixx: moment of inertia about the x-axis (roll)       --- (scalar)
  ks: spring stiffness of suspension                   --- (scalar)
  ls: distance between suspension fixtures to vehicle  --- (scalar)
  g: gravity                                           --- (scalar)
  bs: suspension damping coefficient                   --- (scalar)
  iz: vehivle moment of inertia about z axis (yaw)     --- (scalar)
  kus_d: desired understeer coefficient of vehicle     --- (scalar)
  rmax: maximum allowed yaw rate (to be selected)      --- (scalar)
  Ts: sampling time for euler discretization           --- (scalar)
  Q: cost matrix for states                            --- (4x4)
  R: cost matrix for inputs                            --- (4x4)
  W0: initial torques and steering angles [Q1,delta1,Q2,delta2,Q3,delta3,Q4,delta4] ---(8x1)
  Phir: banking angle at over the prediction horizon   --- (Nx1)
  alphamax: maximum allowable rear wheel side slip     --- (scalar)
  Qmax: maximum torque attainable at each wheel        --- (scalar)
  deltamax: maximum steering angle                     --- (scalar)
  x0: initial states [y,vy,psi,r,phi,phid]             --- (6x1)
  psid: desired heading angle                          --- (scalar)
  yd:desired lateral position                          --- (scalar)
  N: MPC Prediction horizon                            --- (scalar)
  """
  # Tire model Linearization 
  fy1, ca1, a1 = DugoffLinearizer(slip_angle[0],slip_ratio[0], ca[0], cs[0], mux, fz0[0])
  fy2, ca2, a2 = DugoffLinearizer(slip_angle[1],slip_ratio[1], ca[1], cs[1], mux, fz0[1])
  fy3, ca3, a3 = DugoffLinearizer(slip_angle[2],slip_ratio[2], ca[2], cs[2], mux, fz0[2])
  fy4, ca4, a4 = DugoffLinearizer(slip_angle[3],slip_ratio[3], ca[3], cs[3], mux, fz0[3])  

  B11 = np.array([[0,0,0,0,0,0], [0,-ca1/u,0, -lf*ca1/u, 0, 0]])
  B21 = np.array([[1/reff, 0],[0, ca1]])
  D11 = np.array ([0, fy1-ca1*a1])

  B12 = np.array([[0,0,0,0,0,0], [0,-ca2/u,0, -lf*ca2/u, 0, 0]])
  B22 = np.array([[1/reff, 0],[0, ca2]])
  D12 = np.array ([0, fy2-ca2*a2])

  B13 = np.array([[0,0,0,0,0,0], [0,-ca3/u,0, lr*ca3/u, 0, 0]])
  B23 = np.array([[1/reff, 0],[0, ca3]])
  D13 = np.array ([0, fy3-ca3*a3])

  B14 = np.array([[0,0,0,0,0,0], [0,-ca4/u,0, lr*ca4/u, 0, 0]])
  B24 = np.array([[1/reff, 0],[0, ca4]])
  D14 = np.array ([0, fy4-ca4*a4])

  #Local actuator reconfiguration matrix: 0 inactive actuator 1 active actuator
  tq1 = 0
  tq2 = 0
  tq3 = 1
  tq4 = 1
  tdelta1 = 1
  tdelta2 = 1
  tdelta3 = 0
  tdelta4 = 0

  Tw1 = np.array([[tq1,0],[0,tdelta1]])
  Tw2 = np.array([[tq2,0],[0,tdelta2]])
  Tw3 = np.array([[tq3,0],[0,tdelta3]])
  Tw4 = np.array([[tq4,0],[0,tdelta4]])

  #Reconfiguration (Mapping) Matricies: Delta1 input
  Lw1 = np.array ([[np.cos(delta1), -np.sin(delta1)],[np.sin(delta1), np.cos(delta1)]])
  Lw2 = np.array ([[np.cos(delta1), -np.sin(delta1)],[np.sin(delta1), np.cos(delta1)]])
  Lw3 = np.array([[1,0],[0,1]])
  Lw4 = np.array([[1,0],[0,1]])

  #Mapping Matrix from corner forces
  Lc = np.array([[1,0,1,0,1,0,1,0],[0,1,0,1,0,1,0,1], [-tf/2,lf,tf/2,lf,-tr/2,-lr,tr/2,-lr]])

  # Vehicle Body Dynamics y vy psi,psidot roll, rollrate 
  # Equations written in block 1 below
  den =2*(ms**2*hs**2+m*ms*hs**2+m*ixx)
  Af = np.array([[0,1,0,0,0,0], [0,0,0,-u, (hs*ks*ls**2*ms-2*g*hs**2*ms**2)/den, bs*hs*ls**2*ms/den], [0,0,0,1,0,0],[0,0,0,0,0,0],[0,0,0,0,0,1],[0,0,0,0,(2*g*hs*m*ms-ks*ls**2*m)/den, -bs*ls*ls*m/den]])
  Bf = np.array([[0,0,0],[0,(ixx+ms*hs**2)/den, 0],[0,0,0],[0,0,1/iz],[0,0,0],[0,2*ms*hs/den,0]])
  Cpr = np.array([[0],[(2*g*ms**2*hs**2-hs*ks*ls**2*ms)/den],[0],[0],[0],[(ks*ls**2*m-2*g*hs*m*ms)/den]])
  
  # X_dot = AX+EW+BU+D+FPhir
  # W = wheel torque, steer angle for each wheel (8x1)
  # States are v r phi, phi_dot (ignoring tyre dynamics)
  # U = change in wheel torque, change in steering angle for each tire

  Tw = block_diag(Tw1,Tw2,Tw3,Tw4)
  Lw = block_diag(Lw1, Lw2, Lw3, Lw4)
  B1 = np.hstack((B11.T, B12.T, B13.T, B14.T)).T
  B2 = block_diag(B21, B22, B23, B24)
  D1 = np.hstack((D11,D12,D13,D14)).T
  
  A = Af+Bf@Lc@Lw@B1
  E = Bf@Lc@Lw@B2
  B = Bf@Lc@Lw@B2@Tw
  D = Bf@Lc@Lw@D1
  D = D.reshape(6,1)
  F = Cpr

###############################################################################
  ## CONTROL OBJECTIVES ##
  rmax = muy*g/u
  l= lf+lr
  rb = u/(l+kus_d*u**2)*delta1
  rd = np.sign(delta1)*np.min([np.abs(rb), rmax])
  xd = np.array([yd, 0, psid, rd[0], 0 ,0])
  xd = xd.reshape(6,1)
  
  nX = 6
  nU = 8
  nW = 8
  nphir = 1

  # In the paper they use zero order hold, I will use euler discretization
  Ad = Ts*A + np.eye(np.size(A,0))
  Bd = Ts*B
  Ed = Ts*E
  Dd = Ts*D
  Fd = Ts*F
  

  #Formulating Problem:
  Sx, Su, Sw, Sd, Sf = Sx_Su_Sw_Sd_Sf(Ad, Bd, Ed, Dd, Fd, N) ## Check blocks 2 and 3 below for explanation

  # We need a weighting matrix Q. I would say emphasis on tracking lateral position, then heading, then lateral velocity, with little concern for roll charact.
  PN = Q.reshape(6,)
  Q_temp = np.kron(np.eye(N),np.diag(Q.reshape(6,)))
  Qbar  = block_diag(Q_temp,np.diag(PN))
  ##R = np.array(np.ones((8,1)))
  ##R_temp = np.kron(np.eye(N), np.diag(R))
  Rbar = np.eye(nU*N)
  ##Rbar = block_diag(R_temp)

  Wbar = np.kron(np.ones((1,N)), W0).T #W0 recieved from vehicle, vector of inputs at time zero #### check
  Xdbar = np.kron(np.ones((1,N+1)), xd.T).T


  # Check block 4
  H = Su.T@Qbar@Su + Rbar
  F1 = Sx.T @Qbar @ Su
  F2 = Sw.T @ Qbar @ Su
  F3 = Sd.T@Qbar@Su
  F4 = - Qbar@Su
  F5 = Sf.T@Qbar@Su

  # Finally Cost Matricies:

  P = H
  Phirbar = np.kron(np.ones((1,N)), Phir)

  q = x0.T@F1+Wbar.T@F2 +F3+ Phirbar@F5+ Xdbar.T@F4 #x0 from vehicle, xdes from path planner

   
  
  Ax= np.array([[0,0,0,1,0,0],[0,0,0,-1,0,0],[0,-1/u,0,lr/u,0,0],[0,1/u,0,-lr/u,0,0]])
  bx = np.array([rmax, rmax, alphamax,alphamax]).reshape(4,1)

  fx0_p=np.zeros((4,1))
  for i in range(4):
    fx0_p[i] = mux*fz0[i]*np.sqrt(1-(fy0[i]/(muy*fz0[i]))**2)


  Au = np.array([[1,0,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[0,0,0,0,1,0,0,0],[0,0,0,0,0,0,1,0]])
  Au2 = np.array([[0,1,0,0,0,0,0,0],[0,0,0,1,0,0,0,0],[0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,1]])
  Au3 = np.array([[0,1,0,-1,0,0,0,0],[0,-1,0,1,0,0,0,0]])
  Au4 = np.array([[0,0,0,0,1,0,-1,0],[0,0,0,0,-1,0,1,0]])
  Au = np.vstack((Au,-Au,Au,-Au,Au2,-Au2,Au3,Au4))


  bu_fp = fx0_p.T-np.array([W0[0],W0[2],W0[4],W0[6]])
  bu_fp = bu_fp.reshape(4,1)
  bu_sp = fx0_p.T+np.array([W0[0],W0[2],W0[4],W0[6]])
  bu_sp = bu_sp.reshape(4,1)
  butp = Qmax*np.array([0,0,1,1]) - np.array([W0[0],W0[2],W0[4],W0[6]])
  butp = butp.reshape(4,1)
  bufp = Qmax*np.array([0,0,1,1]) + np.array([W0[0],W0[2],W0[4],W0[6]])
  bufp = bufp.reshape(4,1)
  bufip =deltamax*np.array([1,1,0,0])-np.array([W0[1],W0[3],W0[5],W0[7]])
  busip =-deltamax*np.array([1,1,0,0])-np.array([W0[1],W0[3],W0[5],W0[7]])
  bufip = bufip.reshape(4,1)
  busip = busip.reshape(4,1)
  bu = np.vstack((bu_fp,bu_sp,butp,bufp,bufip,busip,0.01,0.01,0.01,0.01))

  
  # Check Block 5
  G0_firstpart= block_diag(np.kron(np.eye(N),Au))
  
  temp = np.kron(np.ones((N+1,N+1)), Ax)
  G0_sp = temp@Su
  G0 = np.vstack((G0_firstpart,G0_sp))


  w0 = np.vstack((np.kron(np.ones((N,1)),bu), np.kron(np.ones((N+1,1)),bx)))


  e0_sp = -Ax
  Ax_tmp = -Ax@A
  for i in range(N):
      e0_sp = np.vstack((e0_sp, Ax_tmp))
      Ax_tmp = Ax_tmp @ A
  e0_fp = np.zeros((N*26, 6)) ##2 length of Bu *n
  
  E0 = np.vstack((e0_fp,e0_sp))

  A = G0
  b = E0@x0+w0

  
  # Optimizing with CasADI
  qp ={}
  H  = 2*cas.DM(H)
  #g = cas.DM(q)

  uba = cas.DM(b.astype(float))
  A = cas.DM(A.astype(float))
  A = A.reshape((A.size1(),H.size1()))

  qp ={}
  qp['h']= H.sparsity()
  qp['a']=A.sparsity()
  S = cas.conic('S','qrqp',qp)

  r = S(h = H, \
        a =A, uba = uba)
  uOpt = r['x']
  JOpt = r['cost']
  return uOpt, JOpt

## Block 1: Vehicle Dynamics Equations (Derived)
Assumptions: the roll angle and bank angle difference is small

\begin{aligned}
\frac{d}{dt}\begin{bmatrix}
y \\
v_y \\
\psi \\
r \\ 
\phi \\
\dot{\phi}\\
\end{bmatrix} = 
\begin{bmatrix}
0 & 1 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & -u & \frac{h_s k_s l_s^2 m_s -2g h_s^2 m_s^2}{2(m_s^2 h_s^2 - h_s k_s l_s^2 m_s)} &\frac{b_s h_s l_s^2 m_s}{2(m_s^2 h_s^2 - h_s k_s l_s^2 m_s)}\\
0 & 0 & 0 &1 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 \\ 
0 & 0 & 0 & 0 & \frac{2 g h_s m m_s - k_s l_s^2 m}{2(m_s^2 h_s^2 - h_s k_s l_s^2 m_s)} & \frac{-b_s l_s^2 m}{2(m_s^2 h_s^2 - h_s k_s l_s^2 m_s)}
\end{bmatrix}
\begin{bmatrix}
y \\
v_y \\
\psi \\
r \\ 
\phi \\
\dot{\phi}\\
\end{bmatrix} +\begin{bmatrix}
0 & 0 &0\\
0 & \frac{I_xx+m_s h_s^2}{2(m_s^2 h_s^2 - h_s k_s l_s^2 m_s)} & 0\\
0 & 0 & 0 \\
0 & 0 & \frac{1}{I_z}\\
0 & 0 & 0\\
0 & \frac{2 m_s h_s}{2(m_s^2 h_s^2 - h_s k_s l_s^2 m_s)} & 0
\end{bmatrix}
\begin{bmatrix}
F_x \\
F_y \\
M_z
\end{bmatrix} + \begin{bmatrix}
0 \\
\frac{2 g m_s^2 h_s^2 - h_s k_s l_s^2 m_s}{2(m_s^2 h_s^2 - h_s k_s l_s^2 m_s)} \\
0 \\
0 \\
0 \\
\frac{k_s l_s^2 m - 2 g h_s m m_s}{2(m_s^2 h_s^2 - h_s k_s l_s^2 m_s)}
\end{bmatrix} \phi_r 
\end{aligned} \\

\begin{aligned}
\dot{X_b}= A_F X_b + B_F F +C_\phi \phi_r
\end{aligned}



With substitution of the matrices required for mapping we get the following:

$$
\dot{X}_b=\left(A_F+B_F L_c L_w B_1\right) X_b+\left(B_F L_c L_w B_2\right) W+\left(B_F L_c L_w B_2 T_w\right) U+B_F L_c L_w D_1 +C_\phi \phi_r
$$

$$
\begin{aligned}
&W=\left[Q_1 \delta_1 Q_2 \delta_2 Q_3 \delta_3 Q_4 \delta_4\right]^T \\
&U=\left[\Delta Q_1 \Delta \delta_1 \Delta Q_2 \Delta \delta_2 \Delta Q_3 \Delta \delta_3 \Delta Q_4 \Delta \delta_4\right]^T \\
&
\end{aligned}
$$

## Block 2: In QP format, the problem has the following formulation
$$
\begin{aligned}
&{\left[\begin{array}{c}
X(0) \\
X^1 \\
\vdots \\
\vdots \\
X^N
\end{array}\right]=\left[\begin{array}{c}
I \\
A_d \\
\vdots \\
\vdots \\
A_d^N
\end{array}\right] X(0)+\left[\begin{array}{cccc}
0 & \ldots & 0 \\
B_d & 0 & \ldots & 0 \\
A_d B_d & \ddots & \ddots & \vdots \\
\vdots & \ddots & \ddots & \vdots \\
A_d^{N-1} B_d & \ldots & B_d
\end{array}\right]\left[\begin{array}{c}
U^0 \\
U^1 \\
\vdots \\
\vdots \\
U^{N-1}
\end{array}\right]} \\
&+\left[\begin{array}{cccc}
0 & \ldots & 0 \\
E_d & 0 & \ldots & 0 \\
A_d E_d & \ddots & \ddots & \vdots \\
\vdots & \ddots & \ddots & \vdots \\
A_d^{N-1} E_d & \ldots & \ldots & E_d
\end{array}\right]\left[\begin{array}{c}
W^0 \\
W^0 \\
\vdots \\
\vdots \\
W^0
\end{array}\right]+\left[\begin{array}{c}
0 \\
D_d \\
A_d D_d \\
\vdots \\
A_d^{N-1} D_d
\end{array}\right] \\
&{+\left[\begin{array}{cccc}
0 & \ldots & 0 \\
F_d & 0 & \ldots & 0 \\
A_d F_d & \ddots & \ddots & \vdots \\
\vdots & \ddots & \ddots & \vdots \\
A_d^{N-1} F_d & \ldots & F_d
\end{array}\right]\left[\begin{array}{c}
\phi_r^0 \\
\phi_r^1 \\
\vdots \\
\vdots \\
\phi_r^{N-1}
\end{array}\right]} \\
&
\end{aligned}
$$

## Block 3: Where we define the following (as per the function Sx_Su_Sw_Sd_Sf)

\begin{aligned}
&\bar{X}=\left[\begin{array}{c}
X(0) \\
X^1 \\
\vdots \\
\vdots \\
X^N
\end{array}\right], \quad \bar{U}=\left[\begin{array}{c}
U^0 \\
U^1 \\
\vdots \\
\vdots \\
U^{N-1}
\end{array}\right], \quad \bar{W}=\left[\begin{array}{c}
W^0 \\
W^0 \\
\vdots \\
\vdots \\
W^0
\end{array}\right] \\
&S_w=\left[\begin{array}{cccc}
0 & \ldots & \ldots & 0 \\
E_d & 0 & \ldots & 0 \\
A_d E_d & \ddots & \ddots & \vdots \\
\vdots & \ddots & \ddots & \vdots \\
A_d^{N-1} E_d & \ldots & E_d
\end{array}\right], \quad S_d=\left[\begin{array}{c}
0 \\
D_d \\
A_d D_d \\
\vdots \\
A_d^{N-1} D_d
\end{array}\right] \\
&
\end{aligned}





$$
S_x=\left[\begin{array}{c}
I \\
A_d \\
\vdots \\
\vdots \\
A_d^N
\end{array}\right], \quad S_u=\left[\begin{array}{cccc}
0 & \ldots & 0 \\
B_d & 0 & \ldots & 0 \\
A_d B_d & \ddots & \ddots & \vdots \\
\vdots & \ddots & \ddots & \vdots \\
A_d^{N-1} B_d & \ldots & B_d
\end{array}\right], \quad S_f=\left[\begin{array}{cccc}
0 & \ldots & 0 \\
F_d & 0 & \ldots & 0 \\
A_d F_d & \ddots & \ddots & \vdots \\
\vdots & \ddots & \ddots & \vdots \\
A_d^{N-1} F_d & \ldots & F_d
\end{array}\right]
$$



## Block 4: Cost function
$$
\begin{gathered}
J=\frac{1}{2} \bar{U}^T H \bar{U}+\left(X(0)^T F_1+\bar{W}^T F_2+F_3+\bar{X}_d^T F_4+\bar{\phi}_r^T F_5\right) \bar{U}+\text { Const. } \\
H=S_u^T \bar{Q} S_u+\bar{R} \\
F_1=S_x^T \bar{Q} S_u \\
F_2=S_w^T \bar{Q} S_u \\
F_3=S_d^T \bar{Q} S_u \\
F_4=-\bar{Q} S_u\\
F_5= S_f^T \bar{Q} S_u
\end{gathered}
$$

## Block 5: Linear constraint formulation
$$
\mathcal{X}=\left\{x \mid A_x x \leq b_x\right\} \quad \mathcal{U}=\left\{u \mid A_u u \leq b_u\right\} \quad \mathcal{X}_f=\left\{x \mid A_f x \leq b_f\right\}
$$
Then $G_0, E_0$ and $w_0$ are defined as follows
$$
G_0=\left[\begin{array}{cccc}
A_u & 0 & \cdots & 0 \\
0 & A_u & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & A_u \\
0 & 0 & \cdots & 0 \\
A_x B & 0 & \cdots & 0 \\
A_x A B & A_x B & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
A_f A^{N-1} B & A_f A^{N-2} B & \ldots & A_f B
\end{array}\right], E_0=\left[\begin{array}{c}
0 \\
0 \\
\vdots \\
0 \\
-A_x \\
-A_x A \\
-A_x A^2 \\
\vdots \\
-A_f A^N
\end{array}\right], w_0=\left[\begin{array}{c}
b_u \\
b_u \\
\vdots \\
b_u \\
b_x \\
b_x \\
b_x \\
\vdots \\
b_f
\end{array}\right]
$$

In [12]:
# States: y,v,psi, r, phi, phi_dot
# Inputs: Change in each wheel torque and in each wheel steer angle

# Vehicle Parameters: 
reff = 0.1905
lf = 1.6566
lr = 1.3152
l = lf+lr
tf = 1.638762
tr = 1.5239686
m = 803.182 #kg
mf = 782/2.2
mr = 985/2.2
mufw = 29.5
murw = 36
ms = m - 2*(mufw+murw)
ixx = 200 
hs = 0 #considered CG and roll center to be same point ##############
iy = 800
iz = 1200
g = 9.81

delta_d = 0####
cs = 80000*np.ones((4,1)) ####
ca = 47275*np.ones((4,1)) #####
delta = np.array([0,0,0,0])
x0 = np.array([0,0,0,0,0,0])
slip_angle = np.array([0,0,0,0])
slip_ratio = np.array([0.1,0.1,0.1,0.1])
fz0 = m*g/4*np.ones((4,1))
W0 = np.array([0,0,0,0,0,0,0,0])
fy0 = np.zeros((4,1))
kus_d = mf /ca[0] +mr /cs[0] #Caf and Car needed
mux = 1
muy =1
u = 20
delta1 = 0
ks = 10000
ls =1
bs = 1000
rmax = 0.1
Ts = 0.1
Q =np.ones((6,1))
R = np.ones((8,1))
Phir = 0
alphamax = 0.2
Qmax = 1000
deltamax = 0.2
psid = 0
yd = 0
N = 1
#################################################################################
nx = 1
nu = 8

M = 1   # Simulation time

xOpt = np.zeros((nx, M+1))
uOpt = np.zeros((nu, M))


xPred = np.zeros((nx, N+1, M))

feas = np.zeros((M, ), dtype=bool)

for t in range(M):
    uopt,jopt = CFTOC_VD(slip_angle, slip_ratio, ca, cs, mux,muy, fz0,fy0, u,reff,delta1,tf,tr,lf,lr,ms,hs,m,ixx,ks,ls,g,bs,iz,kus_d,rmax,Ts,Q,R,W0,Phir,alphamax,Qmax,deltamax,x0,psid,yd,N)    
      
    JO = jopt
    print(uopt[:,0])
    print(np.shape(uopt))


c:\Users\jadra\.conda\envs\MPC_IAC\lib\site-packages\numpy\core\fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 90 is different from 108)